In [1]:
# Import des paquets nécessaires
from typing import Any, List, NoReturn

from bdsim import TransferBlock
import control as ctrl
import ipywidgets as widgets
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import system

# Pendule

## Description du système

<img src="img/systeme.svg"  height="100%" align="right">

Considérons un pendule de masse $m$ \[kg\] et de longueur $l$ \[m\] connecté à un pivot par une de ses extrémités. Un couple $\tau$ \[Nm\] est appliqué au niveau du pivot, ainsi qu'un couple d'amortissement caractérisé par $-b \dot{\theta}$ \[Nm\], où $b$ est le coefficient d'amortissement \[Nms\] et $\dot{\theta}$ la vitesse angulaire du pendule \[rad/s\].

On cherche à contrôler la position en angle du pendule.

## Équation du mouvement

Pour définir l'équation du mouvement du pendule, on choisit un repère orthonormé dont l'origine se trouve sur le pivot. Dès lors, on peut écrire la position du centre de masse du pendule $\pmb{r}_\text{cm}$ \[m\] comme
$$
\pmb{r}_\text{cm} = \frac{l}{2}
    \begin{bmatrix}
        \sin(\theta) \\
        -\cos(\theta) \\
        0
    \end{bmatrix},
$$
où $\theta$ est mesuré dans le sens trigonométrique par rapport à la position d'équilibre stable du pendule, à savoir sa position verticale sous le pivot.

En dérivant cette expression, on obtient la vitesse du centre de masse $\pmb{v}_\text{cm}$ \[m/s\] telle que
$$
\pmb{v}_\text{cm} = \frac{l}{2} \dot{\theta}
\begin{bmatrix}
    \cos(\theta) \\
    \sin(\theta) \\
    0
\end{bmatrix}.
$$

On définit aussi la vitesse angulaire du pendule autour de son centre de masse $\pmb{\omega}$ \[rad/s\] qui s'exprime par
$$
\pmb{\omega} = \begin{bmatrix}
    0 \\
    0 \\
    \dot{\theta}
\end{bmatrix},
$$
et la matrice d'inertie du pendule $J$ telle que
$$
J = \begin{bmatrix}
    J_x & -J_{xy} & -J_{xz} \\
    -J_{xy} & J_y & -J_{yz} \\
    -J_{xz} & -J_{yz} & J_z
\end{bmatrix}.
$$

Dès lors, on peut définir l'énergie cinétique du pendule $E_c$ \[J\] grâce à l'expression
$$
\begin{align*}
E_c 
    &= \frac{m}{2} \pmb{v}_\text{cm}^\top \pmb{v}_\text{cm} + \frac{J}{2} \pmb{\omega}^T \pmb{\omega} = \frac{m}{2} \pmb{v}_\text{cm}^\top \pmb{v}_\text{cm} + J_z \dot{\theta}^2 \\
    &= \frac{m}{2}\frac{l^2}{4} \dot{\theta}^2 \left( \cos(\theta)^2 + \sin(\theta)^2 \right) + \frac{1}{2} \frac{ml^2}{12} \dot{\theta}^2 \\
    &= \frac{ml^2}{6} \dot{\theta}^2.
\end{align*}
$$

De la même manière, si on définit que l'énergie potentielle $E_p$ \[J\] du pendule est nulle dans sa position d'équilibre stable, on peut écrire l'expression
$$
\begin{align*}
E_p 
    &= mg \left( y - y_0 \right) \\
    &= mg \left( -\frac{l}{2}\cos(\theta) + \frac{l}{2} \cos(0) \right) \\
    &= \frac{mgl}{2} \left(1 - \cos(\theta) \right).
\end{align*}
$$

Grâce à ces deux expressions, on peut déterminer le Lagrangien
$$
L = E_c - E_p = \frac{ml^2}{6} \dot{\theta}^2 - \frac{mgl}{2} \left(1 - \cos(\theta) \right).
$$
En utilisant la méthode d'Euler-Lagrange, on peut donc écrire
$$
\frac{\partial}{\partial t} \left( \frac{\partial L}{\partial \dot{\theta}} \right) - \frac{\partial L}{\partial \theta} = \tau - b \dot{\theta},
$$
avec
$$
\begin{align*}
    \frac{\partial L}{\partial \dot{\theta}} &= \frac{\partial}{\partial \dot{\theta}} \left( \frac{ml^2}{6} \dot{\theta}^2 \right) = \frac{ml^2}{3}\dot{\theta}, \\
    \frac{\partial}{\partial t} \left( \frac{\partial L}{\partial \dot{\theta}} \right) &= \frac{\partial}{\partial t} \left( \frac{ml^2}{3}\dot{\theta} \right) = \frac{ml^2}{3}\ddot{\theta}, \\
    \frac{\partial L}{\partial \theta} &= \frac{\partial}{\partial \theta} \left( \frac{mgl}{2} \cos(\theta) \right) = -\frac{mgl}{2} \sin(\theta),
\end{align*}
$$
qui donne l'équation
$$
\frac{ml^2}{3}\ddot{\theta} + \frac{mgl}{2} \sin(\theta) = \tau - b \dot{\theta}.
$$

## Linéarisation

L'équation du mouvement du pendule n'est pas linéaire à cause de l'effet de la gravité qui varie selon l'angle $\theta$. Pour la linéariser, on définit les conditions d'équilibre pour le système telles que
$$
\theta_e = \text{cste.}, \quad
\dot{\theta}_e = 0, \quad
\ddot{\theta}_e = 0, \quad
\tau_e = \frac{mgl}{2} \sin(\theta),
$$
ainsi que les paramètres linéarisés
$$
\tilde{\theta} = \theta - \theta_e, \quad
\dot{\tilde{\theta}} = \dot{\theta} - \dot{\theta}_e = \dot{\theta}, \quad
\ddot{\tilde{\theta}} = \ddot{\theta} - \ddot{\theta}_e = \ddot{\theta}, \quad
\tilde{\tau} = \tau - \tau_e = \tau - \frac{mgl}{2} \sin(\theta).
$$
On constate donc que
$$
\tau = \tilde{\tau} + \frac{mgl}{2} \sin(\theta),
$$
qui, une fois replacé dans l'équation du mouvement, donne
$$
\frac{ml^2}{3} \ddot{\theta} + \frac{mgl}{2} \sin(\theta) = \tilde{\tau} + \frac{mgl}{2} \sin(\theta) - b \dot{\theta} \Leftrightarrow
\frac{ml^2}{3} \ddot{\theta} = \tilde{\tau} - b \dot{\theta,}
$$
qui est l'expression linéarisée du mouvement du pendule.

## Fonction de transfert

### Pendule

En appliquant la transfomée de Laplace à l'expression linéarisée, on obtient
$$
\frac{ml^2}{3} s^2 \Theta(s) + b s \Theta(s) = \tilde{T}(s),
$$
qui, une fois résolue pour $\Theta(s)$, amène
$$
\Theta(s) = \tilde{T}(s) \left( \frac{1}{\frac{ml^2}{3} s^2 + b s} \right).
$$

On peut en déduire la fonction de transfert du pendule

$$
\frac{\Theta(s)}{\tilde{T}(s)} = \frac{1}{\frac{ml^2}{3} s^2 + b s} = \frac{\frac{3}{ml^2}}{s^2 + \frac{3b}{ml^2} s}.
$$

<p align="center"><img src="img/pendule.svg"  width="400px"></p>


### Gravité

La conséquence de la linéarisation est que cette équation ne tient pas compte de la gravité. Afin d'éviter de vous noyez d'informations, nous verrons plus tard comment ce simulateur la prend en compte pour obtenir une simulation la plus proche de la réalité possible.

## Erreur due à la linéarisation

Il est important de remarquer que la linéarisation du pendule (sans l'action de la gravité) est valable pour n'importe quel angle $\theta$. Cependant, l'approximation faite sur le couple dû à la gravité n'est plus négligeable pour des valeurs de $\theta$ supérieures à quelques degrés.

Il est possible de quantifier l'erreur induite par cette approximation en comparant la valeur réelle de $\sin(\theta)$ à celle de $\tilde{\sin}(\theta) = \theta$.

In [2]:
# Quantification de l'erreur due à la linéarisation
# Définition des widgets
tolerance_slider = widgets.IntSlider(
    min=0,
    max=200,
    step=1,
    value=5,
    description="Tolérance [%]"
)
figure = go.FigureWidget(make_subplots(rows=1, cols=2))
# Définition de l'agencement de la figure
figure["layout"]["xaxis"]["title"] = "Angle [°]"
figure["layout"]["yaxis"]["title"] = "Valeur"
figure["layout"]["xaxis2"]["title"] = "Angle [°]"
figure["layout"]["yaxis2"]["title"] = "Erreur relative [%]"
colors = px.colors.qualitative.Plotly
# Calcul de θ et sin(θ)
π = np.pi
θ = np.linspace(- 1.5 * π / 2, 1.5 * π / 2, 1000)
θ_deg = np.rad2deg(θ)
sin_θ = np.sin(θ)
figure.add_scatter(x=θ_deg, y=sin_θ, row=1, col=1, name="sin(θ)", marker=dict(color=colors[1]))
figure.add_scatter(x=θ_deg, y=θ, row=1, col=1, name="~sin(θ)", marker=dict(color=colors[2]))
# Calcul de l'erreur absolue ε et relative ε_r
ε = np.abs(θ - sin_θ)
ε_r = np.abs(ε / sin_θ) * 100
figure.add_scatter(x=θ_deg, y=ε_r, row=1, col=2, name="ε_r", marker=dict(color=colors[3]))
# Définition de la tolérence
figure.add_shape(type="line", x0=-135, x1=135, y0=0, y1=0, row=1, col=2)
for col, (y0, y1) in [(1, [-3.5, 3.5]), (2, [250, -15])]:
    figure.add_shape(
        type="rect",
        x0=0,
        x1=0,
        y0=y0,
        y1=y1,
        fillcolor=colors[0],
        line=dict(width=0),
        opacity=0.1,
        row=1,
        col=col,
        layer="below"
    )
tolerance_line = list(figure.select_shapes(row=1, col=2))[0]
tolerance_rects = [
    list(figure.select_shapes(row=1, col=col))[-1] for col in [1, 2]
]
figure.add_annotation(x=0, y=0, text="ε_r = 0 % → |θ| ≤ 0.0°", showarrow=False, yshift=15, row=1, col=2)
tolerance_text = list(figure.select_annotations(row=1, col=2))[0]

# Définition de la fonction de rafraîchissement
def update(change: Any = None) -> NoReturn:
    tolerance = tolerance_slider.value
    try:
        θ_deg_limit = -θ_deg[ε_r <= tolerance][0]
    except IndexError:
        θ_deg_limit = 0
    with figure.batch_update():
        tolerance_line.y0 = tolerance_line.y1 = tolerance
        for rect in tolerance_rects:
            rect.x0 = -θ_deg_limit
            rect.x1 = θ_deg_limit
        tolerance_text.text = f"ε_r = {tolerance:3d} % → |θ| ≤ {θ_deg_limit:.1f}°"
        tolerance_text.y = tolerance

update()
tolerance_slider.observe(update, names="value")
widgets.VBox([tolerance_slider, figure])


    'data': [{'marker'…

## Simulation

Sur base des équations du mouvement et des fonctions de transfert développées précédement, il nous est possible de simuler le système de deux manières différentes. La première, réaliste, utilise directement la dynamique du système et ne fait pas d'approximation. Cette méthode est plus précise, mais aussi plus gourmande et a le désavantage de ne pas permettre d'étudier la réponse fréquentielle. La seconde, basée sur le modèle simplifié, est plus rapide et permet d'étudier la réponse fréquentielle.

Nous allons donc, à partir de maintenant, étudier les deux modèles en parallèle afin de pouvoir observer le comportement du système réel en réponse au contrôleur dimensionné sur base du modèle linéarisé. Cela nous permettra aussi de quantifier l'erreur d'approximation.

In [3]:
# Implémentation du pendule dynamique
from typing import Tuple
import matplotlib.animation as animation
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
import pandas as pd
from scipy.interpolate import interp1d

class Plant(TransferBlock):
    """Model the dynamics of a pendulum.

    Model a pendulum as a single rod with its centre of mass placed in the
    middle of the rod. This model accounts for the inertia of the pendulum.

    Parameters
    ----------
    mass : float
        The mass of the pendulum.
    length : float
        The length of the pendulum.
    gravity : float
        The gravity applied to the pendulum.
    initial_state : list[float]
        The initial state of the pendulum defined as
        :math:`x_0 = [\theta_0, \: \dot{\theta}_0]`.
    """
    nin = 1
    nout = 1
    nstates = 2

    def __init__(self, mass: float, length: float, damping: float, gravity: float, initial_state: List[float], **blockargs) -> NoReturn:
        self._x0 = np.array(initial_state).flatten()
        self.mass = mass
        self.length = length
        self.damping = damping
        self.gravity = gravity
        super().__init__(
            self._x0.size,
            inames=["𝛕"],
            snames=["y(t)", "ydot(t)"],
            onames=["y(t)"],
            **blockargs,
        )
    
    def output(self, t: float = 0.0) -> List[float]:
        """Return the angular position of the pendulum."""
        return [self._x[0]]
    
    def deriv(self) -> np.array:
        """Return the differentiated state of the pendulum."""
        theta, theta_dot = self._x
        acceleration = (
            3 / (2 * self.mass * self.length**2) * (
                - 2 * self.damping * theta_dot
                - self.mass * self.gravity * self.length * np.sin(theta)
                + 2 * self.inputs[0]
            )
        )
        try:
            acceleration = acceleration[0]
        except IndexError:
            pass
        return np.array([
            self._x[1],
            acceleration,
        ])

    @property
    def h_dc(self) -> ctrl.TransferFunction:
        """Return the direct chain transfer function of the pendulum."""
        return ctrl.tf(
            3 / (self.mass * self.length**2),
            [1, (3 * self.damping) / (self.mass * self.length**2), 0]
        )

    @property
    def h_r(self) -> ctrl.TransferFunction:
        """Return the return transfer function of the pendulum."""
        return ctrl.tf(self.mass * self.gravity * self.length, 2)
    
    @property
    def h_ol(self) -> ctrl.TransferFunction:
        """Return the open loop transfer function of the pendulum."""
        return self.h_dc * self.h_r
    
    @property
    def h(self) -> ctrl.TransferFunction:
        """Return the full transfer function of the pendulum."""
        return ctrl.feedback(self.h_dc, self.h_r)
    
    def _resample(self, results: pd.DataFrame, dt: float) -> NoReturn:
        """Resample data for constant frame rate."""
        time = np.r_[0, results["t"]]
        rtime = np.arange(0, time[-1], dt)
        self._data = np.zeros((rtime.size, 4))
        self._data[:, 0] = rtime
        self._data[:, 1] = interp1d(time, np.r_[self._x0[0], results["r(t)"]])(rtime)
        self._data[:, 2] = interp1d(time, np.r_[self._x0[0], results["y(t)"]])(rtime)
        self._data[:, 3] = interp1d(time, np.r_[self._x0[0], results["~y(t)"]])(rtime)
    
    def _add_pendulum(self, ax: plt.Axes, color: str, alpha: float = 1.0) -> Tuple[patches.FancyBboxPatch, transforms.Affine2D]:
        thickness = self.length / 10
        pendulum = patches.FancyBboxPatch(
            (-thickness / 2, -thickness / 2),
            self.length + thickness,
            thickness,
            patches.BoxStyle.Round(pad=0, rounding_size=thickness / 2),
            facecolor=color,
            linewidth=0,
            alpha=alpha,
        )
        ax.add_patch(pendulum)
        return pendulum, pendulum.get_transform()

    def _rotate_pendulum(self, pendulum: patches.FancyBboxPatch, initial_transform: transforms.Affine2D, theta: float) -> NoReturn:
        pendulum.set_transform(
            transforms.Affine2D().rotate(theta - np.pi / 2) + initial_transform
        )
    
    def _init(self) -> List[patches.FancyBboxPatch]:
        for pendulum, transform in self._pendulums:
            self._rotate_pendulum(pendulum, transform, self._x0[0])
        return [p for p, t in self._pendulums]
    
    def _update(self, i: int) -> List[patches.FancyBboxPatch]:
        self._rotate_pendulum(*self._pendulums[0], self._data[i, 1])
        self._rotate_pendulum(*self._pendulums[1], self._data[i, 2])
        self._rotate_pendulum(*self._pendulums[2], self._data[i, 1])
        self._rotate_pendulum(*self._pendulums[3], self._data[i, 3])
        return [p for p, t in self._pendulums]
    
    def animate(self, results: pd.DataFrame, dt: float) -> animation.FuncAnimation:
        plt.ioff()
        self._fig, self._axs = plt.subplots(1, 2, figsize=(10, 5))
        plt.close()
        self._fig.tight_layout()
        limits = [-1.1 * self.length, 1.1 * self.length]
        for ax in self._axs:
            ax.set_xlim(limits)
            ax.set_ylim(limits)
            ax.tick_params(axis="both", which="both", length=0, labelsize="small", labelcolor="#2a3f5f")
            for spine in ax.spines.values():
                spine.set_visible(False)
            ax.set_aspect("equal")
            ax.set_facecolor("#e5ecf6")
            ax.grid(True, color="white")
            ax.set_axisbelow(True)
        self._pendulums = [
            self._add_pendulum(self._axs[0], "#636efa", 0.2),  # Ref 1
            self._add_pendulum(self._axs[0], "#ef553b"),  # Reel
            self._add_pendulum(self._axs[1], "#636efa", 0.2),  # Ref 2
            self._add_pendulum(self._axs[1], "#15d09f"),  # Model
        ]
        self._resample(results, dt)
        return animation.FuncAnimation(
            self._fig,
            self._update,
            init_func=self._init,
            frames=self._data.shape[0],
            interval=dt * 1000,
            blit=True
        )

<>:11: SyntaxWarning:

invalid escape sequence '\:'

<>:11: SyntaxWarning:

invalid escape sequence '\:'

C:\Users\P150077\AppData\Local\Temp\ipykernel_11348\2149037807.py:11: SyntaxWarning:

invalid escape sequence '\:'



### Paramètres

Afin de simuler le pendule, vous disposez de nombreux paramètres. Utilisez les widgets ci-dessous pour définir leurs valeurs respectives.

In [17]:
# Définition des paramètres de la consigne
from IPython.display import display, Markdown

shape = widgets.Dropdown(value="step", options=["step", "ramp", "parabole"], description="type", tooltip="Type de consigne")
time = widgets.FloatText(0.0, description="t_0", tooltip="Temps de début de la consigne [s]")
value = widgets.FloatText(1.0, description="consigne", tooltip="Valeur de la consigne en radians")
display(
    Markdown(r"""
#### Définition des paramètres de la consigne
"""),
    widgets.VBox([value])
)


#### Définition des paramètres de la consigne


In [5]:
# Définition des paramètres du pendule
mass = widgets.BoundedFloatText(0.5, min=0.0, description="m", tooltip="Masse du pendule [kg]")
length = widgets.BoundedFloatText(0.3, min=0.0, description="l", tooltip="Longueur du pendule [m]")
gravity = widgets.FloatText(0.0, description="g", tooltip="Gravité appliquée au pendule [m/s²]")
damping = widgets.BoundedFloatText(1e-2, min=0.0, description="b", tooltip="Coefficient d'amortissement du pivot [Nms]")
saturation = widgets.BoundedFloatText(0.0, min=0.0, description="𝛕_lim", tooltip="Couple limite (0 signifie qu'il n'y a pas de limite) [Nm]")
#TODO: Fix initial state using state space for the linearized model
#initial_angle = widgets.FloatText(0.0, description="θ_0", tooltip="Position initiale du pendule [°]")
#initial_speed = widgets.FloatText(0.0, description="θdot_0", tooltip="Vitesse initiale du pendule [rad/s]")
#widgets.VBox([mass, length, gravity, damping, saturation, initial_angle, initial_speed])
display(
    Markdown(r"#### Définition des paramètres du pendule"),
    # widgets.VBox([mass, length, gravity, damping, saturation]))
    widgets.VBox([mass, length, gravity, damping]))

#### Définition des paramètres du pendule

In [6]:
# Définition des paramètres du contrôleur
k_p = widgets.BoundedFloatText(1.0, min=0.0, description="K_p", tooltip="Gain du correcteur proportionnel")
k_i = widgets.BoundedFloatText(0.0, min=0.0, description="K_i", tooltip="Gain de l'intégrateur")
k_d = widgets.BoundedFloatText(0.0, min=0.0, description="K_d", tooltip="Gain du dérivateur")
cutoff = widgets.BoundedFloatText(1e3, min=0.0, description="ω_0", tooltip="Pulsation de coupure du filtre passe-bas du dérivateur")
# display(
#     Markdown(r"#### Définition des paramètres du contrôleur"),
#     widgets.VBox([k_p, k_i, k_d, cutoff]))

In [7]:
# Définition des paramètres de la consigne
from IPython.display import display, Markdown

disturbance_shape = widgets.Dropdown(value="step", options=["step", "sin"], description="type", tooltip="Type de perturbation")
disturbance_time = widgets.FloatText(1.0, description="t_0", tooltip="Temps de début de la perturbation [s]")
disturbance_value1 = widgets.FloatText(0.0, description="v_1", tooltip="Valeur de la perturbation")
disturbance_value2 = widgets.FloatText(0.0, description="v_2", tooltip="Valeur de la perturbation")
# display(
#     Markdown(r"""
# #### Définition des paramètres de la consigne

# Le paramètre `type` détermine la forme du signal de perturbation :
# - Un signal de type `"step"` (échelon) produira une marche de hauteur `v_1`, démarrant à `t_0` et durant `v_2` secondes définie par :  
#     $$
#     r(t) = \begin{cases}
#         0, \: t < t_0 \lor t > t_0 + v_2 \\
#         h, \: t_0 \leq t \leq v_2
#     \end{cases}
#     $$.
# - Un signal de type `"sin"` (sinus) produira une sinusoïde d'amplitude `v_1`, de fréquence `v_2` et démarrant à `t_0` définie par :  
#     $$
#     r(t) = \begin{cases}
#         0, \: t < t_0 \\
#         v_1 \sin(2\pi v_2 (t - t_0)), \: t \geq t_0
#     \end{cases}
#     $$
# """),
#     widgets.VBox([disturbance_shape, disturbance_time, disturbance_value1, disturbance_value2])
# )

In [8]:
# Définition des paramètres du bruit
noise_mean = widgets.BoundedFloatText(0.0, min=0.0, description="Moyenne", tooltip="Moyenne du bruit [°]")
noise_std = widgets.BoundedFloatText(0.0, min=0.0, description="Écart type", tooltip="Écart type du bruit [°]")
# display(
#     Markdown(r"""
# #### Définition des paramètres du bruit

# Le bruit modélisé est dit "gaussien" car il suit une loi normale de `moyenne` $\mu$ \[°\] et d'`écart type` $\sigma$ \[°\]. Il correspond donc au signal :
# $$
# n(t) = \mathcal{N}(\mu, \: \sigma)
# $$
#     """),
#     widgets.VBox([noise_mean, noise_std])
# )

In [9]:
# Définition des paramètres de la simulation
duration = widgets.BoundedFloatText(5.0, min=1.0, description="durée", tooltip="Durée de la simulation [s]")
dt = widgets.BoundedFloatText(0.0, min=0.01, description="Δt", tooltip="Durée minimum d'un pas [s]")
# display(
#     Markdown(r"#### Définition des paramètres de la simulation"),
#     widgets.VBox([duration, dt]))

In [10]:
# Simulation du système
import system
from IPython.display import display, clear_output, Markdown, HTML
import pandas as pd

import ReguLabFct as rlf

import warnings
warnings.filterwarnings("ignore")

def plot_time_response(results: pd.DataFrame) -> go.FigureWidget:
    colors = px.colors.qualitative.Plotly
    figure = go.FigureWidget()
    figure["layout"]["title"] = "Réponse temporelle"
    figure["layout"]["xaxis"]["title"] = "Temps [s]"
    figure["layout"]["yaxis"]["title"] = "Angle [rad]"
    figure.add_scatter(x=results["t"], y=results["r(t)"], name="r(t)")
    figure.add_scatter(x=results["t"], y=results["n(t)"], name="n(t)", line=dict(color=colors[0], dash="dash"), opacity=0.5, visible="legendonly")
    figure.add_scatter(x=results["t"], y=results["d(t)"], name="d(t)", line=dict(color=colors[0], dash="dot"), opacity=0.5, visible="legendonly")
    figure.add_scatter(x=results["t"], y=results["y(t)"], name="y(t)", line=dict(color=colors[1]))
    figure.add_scatter(x=results["t"], y=results["~y(t)"], name="~y(t)", line=dict(color=colors[2]))
    figure.add_scatter(x=results["t"], y=results["~y_r(t)"], name="~y_r(t)", line=dict(color=colors[2]), opacity=0.5, visible="legendonly")
    figure.add_scatter(x=results["t"], y=results["~y_n(t)"], name="~y_n(t)", line=dict(color=colors[2], dash="dash"), opacity=0.5, visible="legendonly")
    figure.add_scatter(x=results["t"], y=results["~y_d(t)"], name="~y_d(t)", line=dict(color=colors[2], dash="dot"), opacity=0.5, visible="legendonly")
    return figure

def plot_time_error(results: pd.DataFrame) -> go.FigureWidget:
    colors = px.colors.qualitative.Plotly
    figure = go.FigureWidget()
    figure["layout"]["title"] = "Erreur temporelle"
    figure["layout"]["xaxis"]["title"] = "Temps [s]"
    figure["layout"]["yaxis"]["title"] = "Angle [rad]"
    figure.add_scatter(x=results["t"], y=results["r(t)"], name="r(t)", visible="legendonly")
    figure.add_scatter(x=results["t"], y=results["r(t)"] - results["y(t)"], name="e(t)", marker=dict(color=colors[1]))
    figure.add_scatter(x=results["t"], y=results["r(t)"] - results["~y(t)"], name="~e(t)", marker=dict(color=colors[2]))
    figure.add_scatter(x=results["t"], y=results["r(t)"] - results["~y_r(t)"], name="~e_r(t)", line=dict(color=colors[2]), opacity=0.5, visible="legendonly")
    figure.add_scatter(x=results["t"], y=results["r(t)"] - results["~y_n(t)"], name="~e_n(t)", line=dict(color=colors[2], dash="dash"), opacity=0.5, visible="legendonly")
    figure.add_scatter(x=results["t"], y=results["r(t)"] - results["~y_d(t)"], name="~e_d(t)", line=dict(color=colors[2], dash="dot"), opacity=0.5, visible="legendonly")
    return figure

def plot_frequency_bode(wo_controller: ctrl.TransferFunction, w_controller: ctrl.TransferFunction) -> go.FigureWidget:
    colors = px.colors.qualitative.Plotly
    figure = rlf.bode([wo_controller, w_controller], margins=True)
    for i, name in enumerate(["P(s)", "C(s).P(s)"]):
        for j in [i, i+2]:
            figure.data[j].line = dict(color=colors[i + 1])
            figure.data[j].name = name
    return go.FigureWidget(figure)

def plot_frequency_nichols(wo_controller: ctrl.TransferFunction, w_controller: ctrl.TransferFunction) -> go.FigureWidget:
    colors = px.colors.qualitative.Plotly
    figure = rlf.nichols([wo_controller, w_controller], margins=True)
    for i, name in enumerate(["P(s)", "C(s).P(s)"]):
        figure.data[i].line = dict(color=colors[i + 1])
        figure.data[i].name = name
    return go.FigureWidget(figure)


def work(change: Any = None) -> NoReturn:
    with out:
        clear_output(False)
        # display(change)
        # Définition du système
        change.icon = "spinner spin"
        change.description = "Définition du système..."
        pendulum = Plant(mass.value, length.value, damping.value, gravity.value, [0, 0], name="plant")
        syst = system.System(
            pendulum,
            dict(shape=shape.value, T=time.value, val=value.value),
            dict(k_p=k_p.value, k_i=k_i.value, k_d=k_d.value, derivator_cutoff=cutoff.value),
            dict(mean=np.deg2rad(noise_mean.value), std=np.deg2rad(noise_std.value)),
            dict(shape=disturbance_shape.value, vals=[disturbance_value1.value, disturbance_value2.value], T=disturbance_time.value),
            saturation=saturation.value,
        )
        # Simulation
        change.description = "Simulation du système..."
        results = syst.simulate(duration=duration.value, dt=dt.value)
        h_plant = syst._blocks["plant"].h._repr_latex_().replace("$", "")
        h_controller = syst._blocks["controller"].h._repr_latex_().replace("$", "")
        h_ol = syst.h_ol._repr_latex_().replace("$", "")
        h = syst.h._repr_latex_().replace("$", "")
        # Animation
        change.description = "Animation du système..."
        anim = pendulum.animate(results, 0.05).to_jshtml()
        clear_output(False)
        # display(change)
        change.description = "Affichage des résultats..."
        display(
            Markdown("""
### Réponse temporelle

Voici les résultats des systèmes réel et linéarisé dans le domaine temporelle.
            """),
            HTML(anim),
            widgets.HBox([plot_time_response(results), plot_time_error(results)]),
            Markdown(f"""
### Réponse fréquentielle

La fonction de transfert du système non contrôlé $P(s)$ et du contrôleur $C(s)$ sont respectivement
$$P(s) = {h_plant} \quad \\text{{et}} \quad C(s) = {h_controller}.$$
Dès lors, la fonction de transfert du système en boucle ouverte $G_\\text{{BO}}(s)$ est donnée par
$$
G_\\text{{BO}}(s) = C(s) \cdot P(s) = {h_controller} \cdot {h_plant} = {h_ol}.
$$
En boucle fermée, la fonction de transfert du système complet $G_\\text{{BF}}(s)$ est donc
$$
G_\\text{{BF}}(s) = \\frac{{C(s) \cdot P(s)}}{{1 + C(s) \cdot P(s)}} = {h}.
$$
Voici les résultats du système linéarisé dans le domaine fréquentiel.
            """),
            widgets.HBox([plot_frequency_bode(syst._blocks["plant"].h, syst.h_ol), plot_frequency_nichols(syst._blocks["plant"].h, syst.h_ol)]),
        )
        change.icon = "rocket"
        change.description = "Simuler"

button = widgets.Button(description="Simuler", icon="rocket", layout=dict(width="100%"))
button.on_click(work)
display(button)

out = widgets.Output()
display(out)

<>:110: SyntaxWarning:

invalid escape sequence '\q'

<>:110: SyntaxWarning:

invalid escape sequence '\q'

<>:110: SyntaxWarning:

invalid escape sequence '\c'

<>:110: SyntaxWarning:

invalid escape sequence '\c'

<>:110: SyntaxWarning:

invalid escape sequence '\c'

<>:110: SyntaxWarning:

invalid escape sequence '\c'

<>:110: SyntaxWarning:

invalid escape sequence '\q'

<>:110: SyntaxWarning:

invalid escape sequence '\q'

<>:110: SyntaxWarning:

invalid escape sequence '\c'

<>:110: SyntaxWarning:

invalid escape sequence '\c'

<>:110: SyntaxWarning:

invalid escape sequence '\c'

<>:110: SyntaxWarning:

invalid escape sequence '\c'

C:\Users\P150077\AppData\Local\Temp\ipykernel_11348\643311580.py:110: SyntaxWarning:

invalid escape sequence '\q'

C:\Users\P150077\AppData\Local\Temp\ipykernel_11348\643311580.py:110: SyntaxWarning:

invalid escape sequence '\q'

C:\Users\P150077\AppData\Local\Temp\ipykernel_11348\643311580.py:110: SyntaxWarning:

invalid escape sequence '\c'

C:\U

Button(description='Simuler', icon='rocket', layout=Layout(width='100%'), style=ButtonStyle())

Output()